# Tensor flow
분산처리 기능이 강화된 (Symbolic Linear Algebra Complier)

## 1. 사용 절차

- 심볼 변수 정의
- 심볼 관계 정의
- 세션 정의 = 함수와 같은 역할
- 세션 사용

### 1-1. 심볼 변수

In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(1.0)
y = tf.Variable(2.0)

### 1-2. 심볼 관계

In [4]:
z = x + y

In [5]:
u = tf.log(z)

### 1-3. 세션
세션 : 계산 그래프를 생성하고 계산 환경 생성

- 세션 생성 : Session
- 세션 사용 : run
- 세션 종료 : close 
* 보통 with를 사용해 Session을 열고 닫는다.

In [7]:
# 세션 생성
sess = tf.Session()

# 초기 값 생성
init = tf.global_variables_initializer()

# 세션 사용
sess.run(init)
print(sess.run(z))
print(sess.run(u))

# 세션 종료
sess.close()

3.0
1.09861


In [8]:
f = x ** 2
fx = tf.gradients(f, [x]) # 미분은 tensor에 있는 것을 사용

# with 문을 통한 Session 열고 닫기
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(f))
    print(sess.run(fx))

1.0
[2.0]


### 1-4. 최적화

Theano의 Update와 달리 GradientDescentOptimizer를 이용해 구현이 가능하다.

## 2. tensorflow로 퍼셉트론 구현

In [9]:
# Load data
from sklearn.datasets import load_iris
iris = load_iris()

idx = np.in1d(iris.target, [0,2])
X_data = iris.data[idx, 0:2]
y_data = (iris.target[idx] - 1.0)[:, np.newaxis]

In [16]:
# tensor flow 구현
np.random.seed(0)

# weight, bias 초기화
w = tf.Variable(1e-3 * np.random.randn(2, 1))
b = tf.Variable(1e-3 * np.random.randn(1)[0])

# Symbol 관계 정의 sigmoid 함수 tanh(wx + b)
d = tf.tanh(tf.matmul(X_data, w) + b)

# Loss function 정의
zero = np.zeros(1, dtype=np.float64)[0]
loss = tf.reduce_sum(tf.maximum(zero, tf.multiply(-y_data, d)))

# optimizer 정의
optimizer = tf.train.GradientDescentOptimizer(1e-7)
train = optimizer.minimize(loss)

In [25]:
# Seesion 생성
with tf.Session() as sess:
    
    # log 생성
    w_summary = tf.summary.histogram("w", w)
    b_summary = tf.summary.histogram("b", b)
    d_summary = tf.summary.histogram("d", d)
    loss_summary = tf.summary.histogram("loss", loss)
    merge = tf.summary.merge([w_summary, b_summary, d_summary, loss_summary])
    
    # 초기화 Session
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(100):
        sess.run(train) # desent 진행
        summary = sess.run(merge) # log 기록
    
    # 결과 확인
    y_pred = sess.run(tf.sign(d))

In [24]:
# 결과 확인
from sklearn.metrics import confusion_matrix
confusion_matrix(y_data, y_pred)

array([[49,  1],
       [ 1, 49]], dtype=int64)